This script gathers perpetual future snapshots from kaiko

In [1]:
import requests
import pandas as pd
import datetime
import time

In [2]:
def process_kaiko(raw):
#trading data
    for i in range(len(raw)):
                    epoch = int(raw[i]['poll_timestamp']/1000)
                    time = datetime.datetime.utcfromtimestamp(epoch)
                
                    mid_price = raw[i]['mid_price']
                    spread = raw[i]['spread']
                    ask_slippage = raw[i]['ask_slippage']
                    bid_slippage = raw[i]['bid_slippage']
                    asks = raw[i]['asks']
                    bids = raw[i]['bids']

                    lst.append([epoch, time, mid_price, spread, ask_slippage, bid_slippage, asks, bids])
    return

In [3]:
symbol = 'doge-usdt'
slippage = 1e6
lst = []

oderbook = 'snapshots'
#oderbook = 'ob_aggregations'
interval = '1h'

#end_time = datetime.datetime.utcnow()
#start_time = end_time - datetime.timedelta(minutes=50)
#end_time_iso = end_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

start_time = datetime.datetime.utcfromtimestamp(1661126400) #22.08.2022
start_time_iso = start_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

In [4]:
counter = 0
cont = True
while cont:
    counter += 1
    header = {'X-Api-Key': '68cf96d421857181898149886cc12ddc'}
    query = f'https://eu.market-api.kaiko.io/v2/data/order_book_snapshots.v1/exchanges/binc/perpetual-future/{symbol}/{oderbook}/full?start_time={start_time_iso}&slippage={slippage}&sort=asc&page_size=100&slippage_ref=best'
    response = requests.get(query, headers=header)
    raw = response.json()
    next_url = raw['next_url']
    raw = raw['data']
    
    print(counter, ': ', start_time_iso, 'to', raw[-1]['poll_date'])
    process_kaiko(raw)
  
    start_time_iso = raw[-1]['poll_date']
    
    #end_time_iso = start_time_iso
    #start_time = start_time - datetime.timedelta(minutes=50)
    #start_time_iso = start_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    
    time.sleep(2)
    
    if int(raw[-1]['poll_timestamp']/1000) >= 1662987600: #today
        cont = False

1 :  2022-08-22T00:00:00.000000Z to 2022-08-22T00:41:26.034Z
2 :  2022-08-22T00:41:26.034Z to 2022-08-22T01:22:41.523Z
3 :  2022-08-22T01:22:41.523Z to 2022-08-22T02:03:55.944Z
4 :  2022-08-22T02:03:55.944Z to 2022-08-22T02:45:11.066Z
5 :  2022-08-22T02:45:11.066Z to 2022-08-22T03:26:25.989Z
6 :  2022-08-22T03:26:25.989Z to 2022-08-22T04:07:40.984Z
7 :  2022-08-22T04:07:40.984Z to 2022-08-22T04:48:56.018Z
8 :  2022-08-22T04:48:56.018Z to 2022-08-22T05:30:11.018Z
9 :  2022-08-22T05:30:11.018Z to 2022-08-22T06:11:25.952Z
10 :  2022-08-22T06:11:25.952Z to 2022-08-22T06:52:41.101Z
11 :  2022-08-22T06:52:41.101Z to 2022-08-22T07:33:55.989Z
12 :  2022-08-22T07:33:55.989Z to 2022-08-22T08:15:11.025Z
13 :  2022-08-22T08:15:11.025Z to 2022-08-22T08:56:25.941Z
14 :  2022-08-22T08:56:25.941Z to 2022-08-22T09:37:42.291Z
15 :  2022-08-22T09:37:42.291Z to 2022-08-22T10:18:56.106Z
16 :  2022-08-22T10:18:56.106Z to 2022-08-22T11:00:13.102Z
17 :  2022-08-22T11:00:13.102Z to 2022-08-22T11:41:25.955Z
18 

140 :  2022-08-25T23:33:55.940Z to 2022-08-26T00:15:11.133Z
141 :  2022-08-26T00:15:11.133Z to 2022-08-26T00:56:26.139Z
142 :  2022-08-26T00:56:26.139Z to 2022-08-26T01:37:41.113Z
143 :  2022-08-26T01:37:41.113Z to 2022-08-26T02:18:55.988Z
144 :  2022-08-26T02:18:55.988Z to 2022-08-26T03:00:11.153Z
145 :  2022-08-26T03:00:11.153Z to 2022-08-26T03:41:26.147Z
146 :  2022-08-26T03:41:26.147Z to 2022-08-26T04:22:41.008Z
147 :  2022-08-26T04:22:41.008Z to 2022-08-26T05:04:20.985Z
148 :  2022-08-26T05:04:20.985Z to 2022-08-26T05:45:35.943Z
149 :  2022-08-26T05:45:35.943Z to 2022-08-26T06:26:50.951Z
150 :  2022-08-26T06:26:50.951Z to 2022-08-26T07:08:05.957Z
151 :  2022-08-26T07:08:05.957Z to 2022-08-26T07:49:21.030Z
152 :  2022-08-26T07:49:21.030Z to 2022-08-26T08:30:36.821Z
153 :  2022-08-26T08:30:36.821Z to 2022-08-26T09:11:50.947Z
154 :  2022-08-26T09:11:50.947Z to 2022-08-26T09:53:06.129Z
155 :  2022-08-26T09:53:06.129Z to 2022-08-26T10:34:21.127Z
156 :  2022-08-26T10:34:21.127Z to 2022-

277 :  2022-08-29T21:46:00.960Z to 2022-08-29T22:27:16.487Z
278 :  2022-08-29T22:27:16.487Z to 2022-08-29T23:08:31.033Z
279 :  2022-08-29T23:08:31.033Z to 2022-08-29T23:49:46.262Z
280 :  2022-08-29T23:49:46.262Z to 2022-08-30T00:31:01.106Z
281 :  2022-08-30T00:31:01.106Z to 2022-08-30T01:12:16.416Z
282 :  2022-08-30T01:12:16.416Z to 2022-08-30T01:53:30.958Z
283 :  2022-08-30T01:53:30.958Z to 2022-08-30T02:34:46.188Z
284 :  2022-08-30T02:34:46.188Z to 2022-08-30T03:16:01.208Z
285 :  2022-08-30T03:16:01.208Z to 2022-08-30T03:57:17.706Z
286 :  2022-08-30T03:57:17.706Z to 2022-08-30T04:38:32.972Z
287 :  2022-08-30T04:38:32.972Z to 2022-08-30T05:19:47.979Z
288 :  2022-08-30T05:19:47.979Z to 2022-08-30T06:01:01.592Z
289 :  2022-08-30T06:01:01.592Z to 2022-08-30T06:42:15.955Z
290 :  2022-08-30T06:42:15.955Z to 2022-08-30T07:23:31.126Z
291 :  2022-08-30T07:23:31.126Z to 2022-08-30T08:04:46.249Z
292 :  2022-08-30T08:04:46.249Z to 2022-08-30T08:46:01.025Z
293 :  2022-08-30T08:46:01.025Z to 2022-

414 :  2022-09-02T19:57:37.176Z to 2022-09-02T20:38:52.432Z
415 :  2022-09-02T20:38:52.432Z to 2022-09-02T21:20:07.452Z
416 :  2022-09-02T21:20:07.452Z to 2022-09-02T22:01:22.286Z
417 :  2022-09-02T22:01:22.286Z to 2022-09-02T22:42:37.197Z
418 :  2022-09-02T22:42:37.197Z to 2022-09-02T23:23:52.262Z
419 :  2022-09-02T23:23:52.262Z to 2022-09-03T00:05:07.247Z
420 :  2022-09-03T00:05:07.247Z to 2022-09-03T00:46:22.274Z
421 :  2022-09-03T00:46:22.274Z to 2022-09-03T01:27:37.325Z
422 :  2022-09-03T01:27:37.325Z to 2022-09-03T02:08:52.196Z
423 :  2022-09-03T02:08:52.196Z to 2022-09-03T02:50:07.490Z
424 :  2022-09-03T02:50:07.490Z to 2022-09-03T03:31:24.214Z
425 :  2022-09-03T03:31:24.214Z to 2022-09-03T04:12:37.433Z
426 :  2022-09-03T04:12:37.433Z to 2022-09-03T04:53:52.166Z
427 :  2022-09-03T04:53:52.166Z to 2022-09-03T05:35:07.146Z
428 :  2022-09-03T05:35:07.146Z to 2022-09-03T06:16:22.173Z
429 :  2022-09-03T06:16:22.173Z to 2022-09-03T06:57:37.225Z
430 :  2022-09-03T06:57:37.225Z to 2022-

551 :  2022-09-06T18:08:53.172Z to 2022-09-06T18:50:07.152Z
552 :  2022-09-06T18:50:07.152Z to 2022-09-06T19:31:22.232Z
553 :  2022-09-06T19:31:22.232Z to 2022-09-06T20:12:38.139Z
554 :  2022-09-06T20:12:38.139Z to 2022-09-06T20:53:52.170Z
555 :  2022-09-06T20:53:52.170Z to 2022-09-06T21:35:07.315Z
556 :  2022-09-06T21:35:07.315Z to 2022-09-06T22:16:22.168Z
557 :  2022-09-06T22:16:22.168Z to 2022-09-06T22:57:37.307Z
558 :  2022-09-06T22:57:37.307Z to 2022-09-06T23:38:56.479Z
559 :  2022-09-06T23:38:56.479Z to 2022-09-07T00:20:07.555Z
560 :  2022-09-07T00:20:07.555Z to 2022-09-07T01:01:22.219Z
561 :  2022-09-07T01:01:22.219Z to 2022-09-07T01:42:38.207Z
562 :  2022-09-07T01:42:38.207Z to 2022-09-07T02:23:52.232Z
563 :  2022-09-07T02:23:52.232Z to 2022-09-07T03:05:07.155Z
564 :  2022-09-07T03:05:07.155Z to 2022-09-07T03:46:22.244Z
565 :  2022-09-07T03:46:22.244Z to 2022-09-07T04:27:37.146Z
566 :  2022-09-07T04:27:37.146Z to 2022-09-07T05:08:52.167Z
567 :  2022-09-07T05:08:52.167Z to 2022-

688 :  2022-09-10T16:20:07.322Z to 2022-09-10T17:01:22.346Z
689 :  2022-09-10T17:01:22.346Z to 2022-09-10T17:42:37.231Z
690 :  2022-09-10T17:42:37.231Z to 2022-09-10T18:23:52.235Z
691 :  2022-09-10T18:23:52.235Z to 2022-09-10T19:05:07.227Z
692 :  2022-09-10T19:05:07.227Z to 2022-09-10T19:46:22.387Z
693 :  2022-09-10T19:46:22.387Z to 2022-09-10T20:27:37.231Z
694 :  2022-09-10T20:27:37.231Z to 2022-09-10T21:08:52.145Z
695 :  2022-09-10T21:08:52.145Z to 2022-09-10T21:50:07.630Z
696 :  2022-09-10T21:50:07.630Z to 2022-09-10T22:31:22.141Z
697 :  2022-09-10T22:31:22.141Z to 2022-09-10T23:12:37.236Z
698 :  2022-09-10T23:12:37.236Z to 2022-09-10T23:53:52.369Z
699 :  2022-09-10T23:53:52.369Z to 2022-09-11T00:35:07.468Z
700 :  2022-09-11T00:35:07.468Z to 2022-09-11T01:16:22.225Z
701 :  2022-09-11T01:16:22.225Z to 2022-09-11T01:57:37.213Z
702 :  2022-09-11T01:57:37.213Z to 2022-09-11T02:38:52.243Z
703 :  2022-09-11T02:38:52.243Z to 2022-09-11T03:20:07.290Z
704 :  2022-09-11T03:20:07.290Z to 2022-

In [5]:
df = pd.DataFrame(lst, columns=['epoch', 'date', 'mid_price', 'spread', 'ask_slippage', 'bid_slippage', 'asks', 'bids'])
df.sort_values(by=['epoch'], ascending=True, inplace=True)
df = df.drop_duplicates(subset=['epoch'], keep='first')
df = df.reset_index(drop=True)
df

,epoch,date,mid_price,spread,ask_slippage,bid_slippage,asks,bids
0,1661126411,2022-08-22 00:00:11,0.069265,0.000009999999999996123,0.0031209304655081334,0.002487313459980781,"[{'amount': '26730.0', 'price': '0.06927'}, {'...","[{'amount': '140367.0', 'price': '0.06926'}, {..."
1,1661126435,2022-08-22 00:00:35,0.069265,0.000009999999999996123,0.0012312409597512405,0.0014153472417334245,"[{'amount': '182344.0', 'price': '0.06927'}, {...","[{'amount': '3439.0', 'price': '0.06926'}, {'a..."
2,1661126460,2022-08-22 00:01:00,0.069275,0.000009999999999996123,0.001460962818932877,0.0014156549980195385,"[{'amount': '70030.0', 'price': '0.06928'}, {'...","[{'amount': '53552.0', 'price': '0.06927'}, {'..."
3,1661126486,2022-08-22 00:01:26,0.069275,0.000009999999999996123,0.0014068846987872789,0.0013093060962444819,"[{'amount': '10401.0', 'price': '0.06928'}, {'...","[{'amount': '245289.0', 'price': '0.06927'}, {..."
4,1661126510,2022-08-22 00:01:50,0.06922500000000001,0.000009999999999996123,0.0016365816733289503,0.0011224616221165338,"[{'amount': '127986.0', 'price': '0.06923'}, {...","[{'amount': '524279.0', 'price': '0.06922'}, {..."
...,...,...,...,...,...,...,...,...
74442,1662987582,2022-09-12 12:59:42,0.06456500000000001,0.000009999999999996123,0.0022241489326453984,0.0019194893392193418,"[{'amount': '50366.0', 'price': '0.06457'}, {'...","[{'amount': '392929.0', 'price': '0.06456'}, {..."
74443,1662987607,2022-09-12 13:00:07,0.064585,0.000009999999999996123,0.0020465899480418253,0.002205938584413406,"[{'amount': '443408.0', 'price': '0.06459'}, {...","[{'amount': '6674.0', 'price': '0.06458'}, {'a..."
74444,1662987632,2022-09-12 13:00:32,0.064595,0.000010000000000010001,0.0020827753039191466,0.0020285876774813368,"[{'amount': '243305.0', 'price': '0.0646'}, {'...","[{'amount': '34681.0', 'price': '0.06459'}, {'..."
74445,1662987657,2022-09-12 13:00:57,0.06453500000000001,0.000009999999999996123,0.0022546641763780623,0.0021412261276906,"[{'amount': '182675.0', 'price': '0.06454'}, {...","[{'amount': '157946.0', 'price': '0.06453'}, {..."


In [6]:
df['ask_slippage'] = df['ask_slippage'].astype('float')
df['bid_slippage'] = df['bid_slippage'].astype('float')
df['spread'] = df['spread'].astype('float')

In [7]:
#store in csv
df.to_csv(f'data/orderbook_doge_usdt.csv', index=False)